In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import sys
import csv
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import ensemble
from sklearn import tree
from sklearn import neural_network
from sklearn import svm
from sklearn import naive_bayes
from sklearn import metrics
from mpl_toolkits.mplot3d import Axes3D
#plt.style.use(style='ggplot')
#plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
path='./cleanData/PER_0/'

In [ ]:
d=[]
length = []
for file in os.listdir(path):
    with open(path+file) as fx:
        reader = csv.reader(fx)
        count = 1
        for line in reader:
            d.append(map(float, line))
            count = count + 1
        length.append(count)

In [ ]:
data = pd.DataFrame()

In [ ]:
data = data.append(d,ignore_index=True)

In [ ]:
data.columns=(['a_LV','v_LV','a_FV','v_FV','SepDist','WarningRange'])   

In [ ]:
data = data.drop(data[data['WarningRange']-data['SepDist'] >= 200].index)

In [ ]:
#multicollinear_corr = np.corrcoef(data, rowvar=0)
#multicollinear_corr

In [ ]:
plt.scatter(data['a_LV'],data['WarningRange']-data['SepDist'])

In [ ]:
data = data[(np.abs(stats.zscore(data))<3).all(axis=1)]

In [ ]:
plt.scatter(data['a_LV'],data['WarningRange']-data['SepDist'])

In [ ]:
data['alert'] = np.where(data['WarningRange'] >= data['SepDist'], 1, 0)

In [ ]:
data[['a_LV','v_LV','a_FV','v_FV','SepDist','WarningRange']] = preprocessing.scale(data[['a_LV','v_LV','a_FV','v_FV','SepDist','WarningRange']],with_mean=True, with_std=True)

In [ ]:
plt.hist(data['a_FV'])

In [ ]:
plt.hist(data['a_LV'])

In [ ]:
plt.hist(data['v_LV'])

In [ ]:
plt.hist(data['v_FV'])

In [ ]:
split1 = int(round(0.6*827))

In [ ]:
split2 = int(round(0.2*827))

In [ ]:
split3 = int(round(0.2*827))

In [ ]:
t1 = sum(length[:split1])

In [ ]:
train = data[:t1] 

In [ ]:
#cv = data[(220163+74426):]

In [ ]:
clf1 = ensemble.RandomForestClassifier(n_estimators = 50, oob_score = True, n_jobs = -1,random_state =50,max_features = "sqrt", min_samples_leaf = 50)
#clf1 = neural_network.MLPClassifier(hidden_layer_sizes=(20,))
clf1 = clf1.fit(train[['a_LV','v_LV','a_FV','v_FV']],train[['alert']])

In [ ]:
#clf2 = ensemble.GradientBoostingClassifier(learning_rate=0.01,n_estimators=50, loss='exponential')
clf2 = tree.DecisionTreeClassifier()
clf2 = clf2.fit(train[['a_LV','v_LV','a_FV','v_FV']],train[['alert']])

In [ ]:
test_tr = list(range(8627,8958))
Fscore1 = []
Fscore2 = []

for i in range(len(test_tr)):
        lenn = test_tr[i] - 8296
        lenn = lenn - 4
        if lenn==823:
            l1 = length[lenn-1]
            test = data[-l1:]
        else:
            l1 = sum(length[:lenn])
            l2 = sum(length[:(lenn+1)])
            test = data[l1:l2]
       
        pred = clf1.predict(test[['a_LV','v_LV','a_FV','v_FV']])
        f1 = metrics.f1_score(pred,test[['alert']],average='micro')
        f1 = round(f1,3)
        Fscore1.append(f1)
        
        pred2 = clf2.predict(test[['a_LV','v_LV','a_FV','v_FV']])
        f2 = metrics.f1_score(pred2,test[['alert']],average='micro')
        f2 = round(f2,3)
        Fscore2.append(f2)

#print('F1-score = ', f1)

In [ ]:
#plt.subplot(2,1,1)
#plt.plot([i for i in range(len(test['WarningRange']))], test['alert'], label="Camp")#, marker='8')
#plt.xlabel("Time step")
#plt.ylabel("CAMP")

#plt.subplot(2,1,2)
#plt.plot([i for i in range(len(test['alert']))], pred, label="Camp")#, marker='8')
#plt.xlabel("Time step")
#plt.ylabel("ML")

#plt.savefig('./camp.png')

In [ ]:
plt.subplot(2,1,1)
plt.title('F1-scores')
plt.hist(Fscore1, label='F1-score from 1st classifier')
plt.legend(loc=2)

plt.subplot(2,1,2)
plt.hist(Fscore2, label='F1-score from 2nd classifier')
plt.legend(loc=2)


In [ ]:
sorted1 = np.sort(Fscore1)
yvals = np.arange(len(sorted1))/float(len(sorted1))
sorted2 = np.sort(Fscore2)
yvals2 = np.arange(len(sorted2))/float(len(sorted2))

c1 = 0.3
c2 = 0.7
x = c1*sorted1 + c2*sorted2
y = c1*yvals+ c2*yvals2

plt.subplot(2,1,1)
plt.plot(sorted1, yvals, color='r',label='1st')
plt.plot(sorted2, yvals2, color='b',label='2nd')
plt.plot(x,y,color='g',label='tree+forest')
plt.title('ECDF')
plt.legend(loc=2)